# 第三章 储存

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、对话缓存储存  ](#二、对话缓存储存--)
     - [2.1 开始对话，第一轮](#2.1-开始对话，第一轮)
     - [2.2 第二轮对话](#2.2-第二轮对话)
     - [2.3 第三轮对话](#2.3-第三轮对话)
     - [2.4 .memory.buffer存储了当前为止所有的对话信息](#2.4-.memory.buffer存储了当前为止所有的对话信息)
     - [2.5 也可以通过memory.load_memory_variables({})打印历史消息](#2.5-也可以通过memory.load_memory_variables({})打印历史消息)
     - [2.6 添加指定的输入输出内容到记忆缓存区](#2.6-添加指定的输入输出内容到记忆缓存区)
 - [三、对话缓存窗口储存](#三、对话缓存窗口储存)
     - [3.1 向memory添加两轮对话，并查看记忆变量当前的记录](#3.1-向memory添加两轮对话，并查看记忆变量当前的记录)
     - [3.2 在看一个例子，发现和上面的结果一样，只保留了一轮对话记忆](#3.2-在看一个例子，发现和上面的结果一样，只保留了一轮对话记忆)
     - [3.3 将对话缓存窗口记忆应用到对话链中](#3.3-将对话缓存窗口记忆应用到对话链中)
 - [四、对话token缓存储存](#四、对话token缓存储存)
     - [4.1 导入相关包和API密钥](#4.1-导入相关包和API密钥)
     - [4.2 限制token数量，进行测试](#4.2-限制token数量，进行测试)
     - [4.3 中文例子](#4.3-中文例子)
 - [五、对话摘要缓存储存](#五、对话摘要缓存储存)
     - [5.1 创建一个长字符串，其中包含某人的日程安排](#5.1-创建一个长字符串，其中包含某人的日程安排)
     - [5.2 基于上面的memory，新建一个对话链](#5.2-基于上面的memory，新建一个对话链)
     - [5.3 中文例子](#5.3-中文例子)


当你与那些语言模型进行交互的时候，他们不会记得你之前和他进行的交流内容，这在我们构建一些应用程序（如聊天机器人）的时候，是一个很大的问题---显得不够智能！因此，在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。

当使用 LangChain 中的储存模块时，它可以帮助保存和管理历史聊天消息，以及构建关于特定实体的知识。这些组件可以跨多轮对话存储信息，并允许在对话期间跟踪特定信息和上下文。LangChain 提供了多种储存类型。其中，缓冲区储存允许保留最近的聊天消息，摘要储存则提供了对整个对话的摘要。实体储存则允许在多轮对话中保留有关特定实体的信息。这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。储存模块可以通过简单的API调用来访问和更新，允许开发人员更轻松地实现对话历史记录的管理和维护。

此次课程主要介绍其中四种记忆模块，其他模块可查看文档学习。
- 对话缓存记忆 (ConversationBufferMemory）
- 对话缓存窗口记忆 (ConversationBufferWindowMemory）
- 对话令牌缓存记忆 (ConversationTokenBufferMemory）
- 对话摘要缓存记忆 (ConversationSummaryBufferMemory）

在LangChain中，Memory指的是大语言模型（LLM）的短期记忆。为什么是短期记忆？那是因为LLM训练好之后（获得了一些长期记忆），它的参数便不会因为用户的输入而发生改变。当用户与训练好的LLM进行对话时，LLM会暂时记住用户的输入和它已经生成的输出，以便预测之后的输出，而模型输出完毕后，它便会“遗忘”之前用户的输入和它的输出。因此，之前的这些信息只能称作为LLM的短期记忆。  
  
为了延长LLM短期记忆的保留时间，则需要借助一些外部存储方式来进行记忆，以便在用户与LLM对话中，LLM能够尽可能的知道用户与它所进行的历史对话信息。 

## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [ ]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q python-dotenv
!pip install -q openai

In [6]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  


## 二、对话缓存储存  
  
这种记忆允许存储消息，然后从变量中提取消息。

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [31]:
# 英文链
llm = ChatOpenAI(temperature=0.0)  #temperature：预测下一个token时，概率越大的值就越平滑(平滑也就是让差异大的值之间的差异变得没那么大)，temperature值越小则生成的内容越稳定
memory = ConversationBufferMemory()
conversation = ConversationChain(   #新建一个对话链（关于链后面会提到更多的细节）
    llm=llm, 
    memory = memory,
    verbose=True   #查看Langchain实际上在做什么，设为FALSE的话只给出回答，看到不到下面绿色的内容
)

In [32]:
# 中文链
llm = ChatOpenAI(temperature=0.0)  #temperature：预测下一个token时，概率越大的值就越平滑(平滑也就是让差异大的值之间的差异变得没那么大)，temperature值越小则生成的内容越稳定
memory_zh = ConversationBufferMemory()
conversation_zh = ConversationChain(   #新建一个对话链（关于链后面会提到更多的细节）
    llm=llm, 
    memory = memory_zh,
    verbose=True   #查看Langchain实际上在做什么，设为FALSE的话只给出回答，看到不到下面绿色的内容
)

### 2.1 开始对话，第一轮

当我们运行predict时，生成了一些提示，如下所见，他说“以下是人类和AI之间友好的对话，AI健谈“等等，这实际上是LangChain生成的提示，以使系统进行希望和友好的对话，并且必须保存对话，并提示了当前已完成的模型链。

In [33]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [34]:
#中文
conversation_zh.predict(input="你好, 我叫皮皮鲁")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫皮皮鲁
AI:

> Finished chain.


'你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？'

### 2.2 第二轮对话

当我们进行下一轮对话时，他会保留上面的提示

In [35]:
conversation.predict(input="What is 1+1?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [36]:
#中文
conversation_zh.predict(input="1+1等于多少？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI:

> Finished chain.


'1+1等于2。'

### 2.3 第三轮对话

为了验证他是否记忆了前面的对话内容，我们让他回答前面已经说过的内容（我的名字），可以看到他确实输出了正确的名字，因此这个对话链随着往下进行会越来越长

In [37]:
conversation.predict(input="What is my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [38]:
#中文
conversation_zh.predict(input="我叫什么名字？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: 我叫什么名字？
AI:

> Finished chain.


'你叫皮皮鲁。'

### 2.4 .memory.buffer存储了当前为止所有的对话信息

In [39]:
print(memory.buffer)   #提取历史消息

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


In [40]:
# 中文
print(memory_zh.buffer)   #提取历史消息

Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？
Human: 1+1等于多少？
AI: 1+1等于2。
Human: 我叫什么名字？
AI: 你叫皮皮鲁。


### 2.5 也可以通过memory.load_memory_variables({})打印历史消息

这里的花括号实际上是一个空字典，有一些更高级的功能，使用户可以使用更复杂的输入，但我们不会在这个短期课程中讨论它们，所以不要担心为什么这里有一个空的花括号。

In [41]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}

In [42]:
# 中文
memory_zh.load_memory_variables({})

{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？\nHuman: 1+1等于多少？\nAI: 1+1等于2。\nHuman: 我叫什么名字？\nAI: 你叫皮皮鲁。'}

### 2.6 添加指定的输入输出内容到记忆缓存区

In [43]:
memory = ConversationBufferMemory()  #新建一个空的对话缓存记忆

In [45]:
memory.save_context({"input": "Hi"},    #向缓存区添加指定对话的输入输出
                    {"output": "What's up"})

In [46]:
print(memory.buffer)   #查看缓存区结果

Human: Hi
AI: What's up
Human: Hi
AI: What's up


In [47]:
memory.load_memory_variables({})  #再次加载记忆变量

{'history': "Human: Hi\nAI: What's up\nHuman: Hi\nAI: What's up"}

In [48]:
#中文
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, 
                    {"output": "你好啊，我叫鲁西西"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西'}

继续添加新的内容，对话历史都保存下来在了！

In [49]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [50]:
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool'}

In [51]:
#中文
memory.save_context({"input": "很高兴和你成为朋友！"}, 
                    {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

当我们在使用大型语言模型进行聊天对话时，**大型语言模型本身实际上是无状态的。语言模型本身并不记得到目前为止的历史对话**。每次调用API结点都是独立的。

聊天机器人似乎有记忆，只是因为通常有快速的代码可以向LLM提供迄今为止的完整对话以及上下文。因此，Memory可以明确地存储到目前为止的所有术语或对话。这个Memory存储器被用作输入或附加上下文到LLM中，以便它可以生成一个输出，就好像它只有在进行下一轮对话的时候，才知道之前说过什么。


## 三、对话缓存窗口储存
  
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。
  
针对以上问题，LangChain也提供了几种方便的memory来保存历史对话。
其中，对话缓存窗口记忆只保留一个窗口大小的对话缓存区窗口记忆。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大

In [52]:
from langchain.memory import ConversationBufferWindowMemory

### 3.1 向memory添加两轮对话，并查看记忆变量当前的记录

In [53]:
memory = ConversationBufferWindowMemory(k=1)     # k=1表明只保留一个对话记忆        

In [54]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [55]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

### 3.2 在看一个例子，发现和上面的结果一样，只保留了一轮对话记忆

In [56]:
#中文
memory = ConversationBufferWindowMemory(k=1)     # k=1表明只保留一个对话记忆        
memory.save_context({"input": "你好，我叫皮皮鲁"}, 
                    {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, 
                    {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

### 3.3 将对话缓存窗口记忆应用到对话链中

In [57]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(  
    llm=llm, 
    memory = memory,
    verbose=False    #这里改为FALSE不显示提示，你可以尝试修改为TRUE后的结果
)

In [61]:
# 中文版
llm = ChatOpenAI(temperature=0.0)
memory_zh = ConversationBufferWindowMemory(k=1)
conversation_zh = ConversationChain(  
    llm=llm, 
    memory = memory_zh,
    verbose=False    #这里改为FALSE不显示提示，你可以尝试修改为TRUE后的结果
)

注意此处！由于这里用的是一个窗口的记忆，因此只能保存一轮的历史消息，因此AI并不能知道你第一轮对话中提到的名字，他最多只能记住上一轮（第二轮）的对话信息

In [58]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [63]:
conversation_zh.predict(input="你好，我是皮皮鲁")

'你好，皮皮鲁！很高兴认识你。我是一个AI助手，可以回答你的问题和提供帮助。有什么我可以帮你的吗？'

In [64]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [65]:
conversation_zh.predict(input="1+1等于几")

'1+1等于2。'

In [66]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

In [67]:
conversation_zh.predict(input="我叫什么名字")

'很抱歉，我无法知道您的名字。'

## 四、对话token缓存储存

使用对话token缓存记忆，内存将限制保存的token数量。如果token数量超出指定数目，它会切掉这个对话的早期部分
以保留与最近的交流相对应的token数量，但不超过token限制。

In [ ]:
#!pip install tiktoken            #需要用到tiktoken包，没有的可以先安装一下

### 4.1 导入相关包和API密钥

In [68]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

llm = ChatOpenAI(temperature=0.0)

### 4.2 限制token数量，进行测试

In [69]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

可以看到前面超出的的token已经被舍弃了！！！

In [70]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### 4.3 中文例子

In [71]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"},  
                    {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"},
                    {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})

{'history': 'AI: 轻舟已过万重山。'}

补充：  

ChatGPT使用一种基于字节对编码（Byte Pair Encoding，BPE）的方法来进行tokenization（将输入文本拆分为token）。  
BPE是一种常见的tokenization技术，它将输入文本分割成较小的子词单元。 

OpenAI在其官方GitHub上公开了一个最新的开源Python库：tiktoken，这个库主要是用来计算tokens数量的。相比较Hugging Face的tokenizer，其速度提升了好几倍 <https://github.com/openai/tiktoken>

具体token计算方式,特别是汉字和英文单词的token区别，参考 <https://www.zhihu.com/question/594159910>


## 五、对话摘要缓存储存

这种Memory的想法是，不是将内存限制为基于最近对话的固定数量的token或固定数量的对话次数窗口，而是**使用LLM编写到目前为止历史对话的摘要**，并将其保存

In [72]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

In [73]:
llm = ChatOpenAI(temperature=0.0)

### 5.1 创建一个长字符串，其中包含某人的日程安排

In [74]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)   #使用对话摘要缓存记忆
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [75]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.'}

### 5.2 基于上面的memory，新建一个对话链

In [76]:
conversation = ConversationChain(  
    llm=llm, 
    memory = memory,
    verbose=True
)

In [77]:
conversation.predict(input="What would be a good demo to show?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'

In [78]:
memory.load_memory_variables({})   #摘要记录更新了

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.\nHuman: What would be a good demo to show?\nAI: A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'}

### 5.3 中文例子

In [79]:
# 创建一个长字符串
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "你好，我叫皮皮鲁"}, 
                    {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, 
                    {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, 
                    {"output": f"{schedule}"})

In [80]:
conversation = ConversationChain(  
    llm=llm, 
    memory = memory,
    verbose=True
)

In [81]:
conversation.predict(input="展示什么样的样例最好呢？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI introduce themselves and become friends. They plan to go on an adventure together. The human asks about their schedule for the day. The AI informs the human about a meeting with their product team at 8 am and the need to prepare a PowerPoint presentation. From 9 am to 12 pm, the human will be busy with LangChain, a useful tool that will help their project progress quickly. At noon, they will have lunch with a customer who has traveled a long way to learn about the latest AI. The AI advises the human to bring their laptop to showcase the latest LLM samples.
Human: 展示什么样的样例最好呢？
AI:

> Finished chain.


'展示一些具有多样性和创新性的样例可能是最好的选择。你可以展示一些基于图像识别的样例，比如人脸识别、物体识别等。另外，你也可以展示一些自然语言处理方面的样例，比如文本生成、情感分析等。最好选择一些能够展示出你们团队的技术实力和创造力的样例。'

In [82]:
memory.load_memory_variables({})   #摘要记录更新了

{'history': "System: The human and AI introduce themselves and become friends. They plan to go on an adventure together. The human asks about their schedule for the day. The AI informs the human about a meeting with their product team at 8 am and the need to prepare a PowerPoint presentation. From 9 am to 12 pm, the human will be busy with LangChain, a useful tool that will help their project progress quickly. At noon, they will have lunch with a customer who has traveled a long way to learn about the latest AI. The AI advises the human to bring their laptop to showcase the latest LLM samples. The human asks what kind of samples would be best to showcase. The AI suggests that showcasing diverse and innovative samples would be the best choice. They recommend demonstrating examples based on image recognition, such as face recognition and object recognition. Additionally, they suggest showcasing examples in natural language processing, such as text generation and sentiment analysis. It is